<a href="https://colab.research.google.com/github/sfarrukhm/making_models_small/blob/main/making_transformers_efficient.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2

In [ ]:
import torch
from pathlib import Path
import numpy as np
from time import perf_counter
from transformers import pipeline
from datasets import load_dataset

clinc = load_dataset("clinc_oos", "plus")

model_checkpoint="transformersbook/bert-base-uncased-finetuned-clinc"
pipeline=pipeline("text-classification",model=model_checkpoint)
intents = clinc["test"].features["intent"]

In [14]:
import evaluate
accuracy_calculate=evaluate.load('accuracy')

In [16]:
class PerformanceBenchmark:
    def __init__(self, pipeline, dataset, optim_type="BERT baseline"):
        self.pipeline=pipeline
        self.dataset = dataset
        self.optim_type = optim_type
        self.intents = dataset.features["intent"]

    def compute_accuracy(self):
        preds, labels=[],[]
        for example in self.dataset:
            pred=self.pipeline(example['text'])[0]['label']
            label=example['intent']
            preds.append(self.intents.str2int(pred))
            labels.append(label)
        accuracy=accuracy_calculate.compute(predictions=preds, references=labels)
        print(f"Accuracy on test set - {accuracy['accuracy']:.3f}")

        return accuracy
    def compute_size(self):
        state_dict = self.pipeline.model.state_dict()
        tmp_path = Path("model.pt")
        torch.save(state_dict, tmp_path)
        # Calculate size in megabytes
        size_mb = Path(tmp_path).stat().st_size / (1024 * 1024)
        # Delete temporary file
        tmp_path.unlink()
        print(f"Model size (MB) - {size_mb:.2f}")

        return {"size_mb": size_mb}
    def time_pipeline(self, query="How to make a bulletproof coffee?"):
        latencies = []
        # Warmup
        for _ in range(10):
             _ = self.pipeline(query)
        # Timed run
        for _ in range(100):
            start_time = perf_counter()
            _ = self.pipeline(query)
            latency = perf_counter() - start_time
            latencies.append(latency)
        # Compute run statistics
        time_avg_ms = 1000 * np.mean(latencies)
        time_std_ms = 1000 * np.std(latencies)
        print(f"Average latency (ms) - {time_avg_ms:.2f} +\- {time_std_ms:.2f}")
        return {"time_avg_ms": time_avg_ms, "time_std_ms": time_std_ms}
    def run_benchmark(self):
        metrics={}
        metrics[self.optim_type]=self.compute_size()
        metrics[self.optim_type].update(self.time_pipeline())
        metrics[self.optim_type].update(self.compute_accuracy())
        return metrics



In [17]:
# Testing the benckmark
pb = PerformanceBenchmark(pipeline, clinc['test'])
performance_metrics=pb.run_benchmark()

Model size (MB) - 418.15
Average latency (ms) - 7.88 +\- 1.07
Accuracy on test set - 0.867


In [11]:
intents = clinc['test'].features["intent"]

In [10]:
intents=clinc['test'].features['intent']
intents.int2str(sample['intent'])

'translate'

In [26]:
import evaluate

# Load the accuracy metric
accuracy_calculate = evaluate.load('accuracy')

# Define predictions and true labels
true = [1,1,0,0]
pred = [0,1,0,0]

# Compute accuracy correctly
result = accuracy_calculate.compute(predictions=pred, references=true)

print(result)  # Output: {'accuracy': some_value}


{'accuracy': 0.75}


In [17]:

def compute_accuracy(self):
    preds, labels=[],[]
    for example in self.dataset:
            pred=self.pipeline(example['text'])[0]['label']
            label=example['intent']
            preds.append(intents.str2int(pred))
            labels.append(label)
    accuracy=accuracy_calculate.compute(predictions=preds, references=labels)
    print(f"Accuracy on test set - {accuracy['accuracy']:.3f}")

    return accuracy
